In [ ]:
import os
import json
import csv
import numpy as np
import pandas as pd
import datetime
import math
import matplotlib.mlab as mlab
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

import tensorflow as tf
# from tensorflow import keras

import shap
import eli5
from eli5.sklearn import PermutationImportance

from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation, LeakyReLU
from keras import optimizers, activations, models
from keras.callbacks import EarlyStopping, TensorBoard, LearningRateScheduler, LambdaCallback, Callback


%matplotlib inline

from sklearn import model_selection, preprocessing, metrics

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
Flattening the JSON blobs

def load_df(csv_path='data/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']

    df = pd.read_csv(csv_path,
                     converters={column: json.loads for column in JSON_COLUMNS},
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)

    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = ["{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print("Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df
%%time
train_df = load_df()
%%time
test_df = load_df("/data/test.csv")
train_df.columns.difference(test_df.columns)
We can see now that:

train.csv has 903,653 rows and 55 columns
test.csv has 804,684 rows and 53 columns
The columns not included in test.csv are totals.transactionRevenue as expected and trafficSource.campaignCode

2. Data Exploration
Now the dataset has been flattened, it is easy to refer to any part of the dataframe and explore various features. Firstly, let's look at the target variable totals.transactionRevenue for each user.

train_df.head()
test_df.head()
There also appear to be many columns with constant information in, such as NaN or not available in demo dataset. If entire columns contain the same information like these then they will likely not be of use. The code below tests for columns that only have 1 unique value.

const_cols = [c for c in train_df.columns if train_df[c].nunique(dropna=False)==1 ]
const_cols
train_df = train_df.drop(const_cols, axis=1)
train_df.shape
test_df = test_df.drop(const_cols, axis=1)
test_df.shape
train_df.to_csv("data/newTrain.csv")
test_df.to_csv("data/newTest.csv")
The columns that contain the null or useless values have been removed from the datasets, which now have 19 fewer columns. I have also saved the new dataframes to new csv files as they will be easier to acquire and work from later.

nTrain_df = pd.read_csv("data/newTrain.csv",
                 dtype={'fullVisitorId': 'str',
                       'campaignCode': 'str'})
print (nTrain_df.shape)
nTrain_df.head()

nTest_df = pd.read_csv("data/newTest.csv",
             dtype={'fullVisitorId': 'str'})
print (nTest_df.shape)
nTest_df.head()
nTrain_df["transactionRevenue"].fillna(0, inplace=True)
nTrain_df["transactionRevenue"] = nTrain_df["transactionRevenue"].astype('float')
gdf = nTrain_df.groupby("fullVisitorId")["transactionRevenue"].sum().reset_index()

plt.figure(figsize=(8,6))
plt.scatter(range(gdf.shape[0]), np.sort(np.log1p(gdf["transactionRevenue"].values)))
plt.xlabel('index', fontsize=12)
plt.ylabel('TransactionRevenue', fontsize=12)
plt.show()

It is clear to see the revenue comes from a select few of customers, as stated by the 80/20 rule

num_buyers = (gdf["transactionRevenue"]>0).sum()
print (num_buyers)
print("Number of unique customers: ",gdf.shape[0]," with non-zero revenue : ", num_buyers, "and the ratio is : ", (num_buyers / gdf.shape[0])*100)
print ("The ratio is much lower than 80/20, only 1.3% of visitors have spent money")


minimum = np.log1p(np.min(gdf["transactionRevenue"]))
maximum = np.log1p(np.max(gdf["transactionRevenue"]))
mean= np.log1p(np.mean(gdf["transactionRevenue"]))
std= np.log1p(np.std(gdf["transactionRevenue"]))

# Show the calculated statistics
print("Statistics for total revenue per fullVisitorID:\n")
print("Minimum: ${}".format(minimum))
print("Maximum: ${}".format(maximum))
print("Mean: ${}".format(mean))
print("Standard deviation: ${}".format(std))
Statistics for total revenue per fullVisitorID:


sigma = math.sqrt(std)
x = np.linspace(mean - 3*sigma, mean + 3*sigma, 100)
plt.plot(x,mlab.normpdf(x, mean, sigma))
plt.show()




def horizontal_bar_chart(y1, x1, y2, x2, y3, x3, data):
    f, axes = plt.subplots(1,3,figsize=(18, 6))

    sns.set(style="whitegrid")
    sns.set_color_codes("pastel")
    sns.barplot(y=y1, x=x1, data=data,
            label="Total", ax=axes[0])

    sns.set(style="whitegrid")
    sns.set_color_codes("pastel")
    sns.barplot(y=y2, x=x2, data=data,
            label="Total", ax=axes[1])

    sns.set(style="whitegrid")
    sns.set_color_codes("pastel")
    sns.barplot(y=y3, x=x3, data=data,
            label="Total", ax=axes[2])

    plt.tight_layout()
data = nTrain_df.groupby("browser")["transactionRevenue"].agg(['size', 'count', 'mean'])
data.columns = ["count", "count of transactions", "mean transaction"]
data.reset_index(inplace=True)
data = data.sort_values(by="count", ascending=False)

horizontal_bar_chart("browser", "count", "browser", "count of transactions", "browser", "mean transaction", data.head(10))

data = nTrain_df.groupby("deviceCategory")["transactionRevenue"].agg(['size', 'count', 'mean'])
data.columns = ["count", "count of transactions", "mean transaction"]
data.reset_index(inplace=True)
data = data.sort_values(by="count", ascending=False)

horizontal_bar_chart("deviceCategory", "count", "deviceCategory", "count of transactions", "deviceCategory", "mean transaction", data)

data = nTrain_df.groupby("operatingSystem")["transactionRevenue"].agg(['size', 'count', 'mean'])
data.columns = ["count", "count of transactions", "mean transaction"]
data.reset_index(inplace=True)
data = data.sort_values(by="count", ascending=False)

horizontal_bar_chart("operatingSystem", "count", "operatingSystem", "count of transactions", "operatingSystem", "mean transaction", data)

data = nTrain_df.groupby("continent")["transactionRevenue"].agg(['size', 'count', 'mean'])
data.columns = ["count", "count of transactions", "mean transaction"]
data.reset_index(inplace=True)
data = data.sort_values(by="count", ascending=False)

horizontal_bar_chart("continent", "count", "continent", "count of transactions", "continent", "mean transaction", data)

data = nTrain_df.groupby("subContinent")["transactionRevenue"].agg(['size', 'count', 'mean'])
data.columns = ["count", "count of transactions", "mean transaction"]
data.reset_index(inplace=True)
data = data.sort_values(by="count", ascending=False)

horizontal_bar_chart("subContinent", "count", "subContinent", "count of transactions", "subContinent", "mean transaction", data)

data = nTrain_df.groupby("country")["transactionRevenue"].agg(['size', 'count', 'mean'])
data.columns = ["count", "count of transactions", "mean transaction"]
data.reset_index(inplace=True)
data = data.sort_values(by="count", ascending=False)

horizontal_bar_chart("country", "count", "country", "count of transactions", "country", "mean transaction", data.head(10))

data = nTrain_df.groupby("region")["transactionRevenue"].agg(['size', 'count', 'mean'])
data.columns = ["count", "count of transactions", "mean transaction"]
data.reset_index(inplace=True)
data = data.sort_values(by="count", ascending=False)

horizontal_bar_chart("region", "count", "region", "count of transactions", "region", "mean transaction", data.head(10))

data = nTrain_df.groupby("source")["transactionRevenue"].agg(['size', 'count', 'mean'])
data.columns = ["count", "count of transactions", "mean transaction"]
data.reset_index(inplace=True)
data = data.sort_values(by="count", ascending=False)

horizontal_bar_chart("source", "count", "source", "count of transactions", "source", "mean transaction", data.head(10))

 
nTrain_df['datetime'] = nTrain_df['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))
nTest_df['datetime'] = nTest_df['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))
def line_plot( data):
    f, axes = plt.subplots(figsize=(18, 6))

    sns.set(style="white")
    sns.lineplot( data=data, palette="tab10", linewidth=2.5)

    plt.tight_layout()
data = nTrain_df.groupby("datetime")["transactionRevenue"].agg(['count'])
line_plot(data)


data = nTrain_df.groupby("datetime")["transactionRevenue"].agg(['size'])
line_plot(data)


data = nTest_df.groupby("datetime")["fullVisitorId"].agg(['size'])
line_plot(data)


# sort by date for train-validation split
nTrain_df.sort_values(by=["datetime"], inplace=True)

# extract sorted dates
dates = pd.DataFrame(nTrain_df["date"])

# set target
target = pd.DataFrame(nTrain_df["transactionRevenue"])

# extract test IDs for later
test_ids = nTest_df["fullVisitorId"].values

# columns to be dropped before training the model
reduce_columns = ["bounces", "Unnamed: 0", "fullVisitorId", "date", "sessionId", "visitId", "visitStartTime", "isMobile", "city",
                 "metro", "networkDomain", "region", "adContent", "adwordsClickInfo.adNetworkType",
                  "adwordsClickInfo.gclId", "adwordsClickInfo.isVideoAd", "adwordsClickInfo.page",
                  "adwordsClickInfo.slot", "campaign", "isTrueDirect", "keyword",
                  "medium", "referralPath", "source", "datetime"]

# drop columns from both datasets, including the two that are missing in the test dataset
train_features = nTrain_df.drop(reduce_columns + ["campaignCode", "transactionRevenue"], axis=1)

test = nTest_df.drop(reduce_columns, axis=1)
target.head()

# One-hot encode data
categorical_features = ["channelGrouping", "browser", "deviceCategory", "operatingSystem",
                        "continent", "country", "subContinent"]
train_features = pd.get_dummies(train_features, columns=categorical_features)

test = pd.get_dummies(test, columns=categorical_features)

# Since the train and test datasets have different number of categories after one-hot encoding they need to be aligned
# an outer join includes all categories from both datasets
train_features, test = train_features.align(test, join='outer', axis=1)
# Replace NaN values throughout dataset
train_features.replace(to_replace=np.nan, value=0, inplace=True)
train_features.shape
(903653, 425)
# Replace NaN values throughout dataset
test.replace(to_replace=np.nan, value=0, inplace=True)
test.shape
(804684, 425)
# Log transform the labels in the target
target = np.log1p(target)
target.head()

# Normalise data
normalized_features = ["visitNumber",  "hits", "newVisits", "pageviews"]

scaler = preprocessing.MinMaxScaler()
train_features[normalized_features] = scaler.fit_transform(train_features[normalized_features])

test[normalized_features] = scaler.fit_transform(test[normalized_features])




def k_fold_split(X, y, dates, num_folds):

    k = int(np.floor(float(X.shape[0]) / num_folds))

    X_train_list = []
    y_train_list = []
    X_val_list = []
    y_val_list = []
    dates_list = []

    for i in range(2, num_folds + 1):

        # percentage split
        split = float(i-1)/i

        x_temp = X[:k*i]
        y_temp = y[:k*i]
        dates_list.append((dates.iloc[k*i]).values)

        # index to split current fold into train and test
        index = int(np.floor(x_temp.shape[0] * split))

        X_train = x_temp[:index]
        y_train = y_temp[:index]

        X_val = x_temp[index:]
        y_val = y_temp[index:]

        X_train_list.append(X_train)
        y_train_list.append(y_train)
        X_val_list.append(X_val)
        y_val_list.append(y_val)

    return X_train_list, y_train_list, X_val_list, y_val_list, dates_list

X_train_list, y_train_list, X_val_list, y_val_list, dates_list = k_fold_split(train_features, target, dates, 5)
sets = [X_train_list, y_train_list, X_val_list, y_val_list, dates_list]

for i in sets:
    for x in i:
        print (x.shape)

from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
new_dates = pd.DataFrame(dates_list)
new_dates['dates'] = new_dates[0].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))
print (new_dates)

dates_long = pd.DataFrame(dates)
dates_long['date'] = dates_long['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))
dates_long.shape
         
def performance_metric(predict, true):
    
    mse = mean_squared_error(predict, true)
    rmse = np.sqrt(mean_squared_error(predict, true))
    return mse, rmse
def train_predict(model, X_train, y_train, X_val, y_val):

    model = model.fit(X_train, y_train)

    val_predictions = model.predict(X_val)

    mse, rmse = performance_metric(val_predictions, y_val)

    return mse, rmse

    print('Validation MSE: %.2f' % mse)
    print('Validation RMSE: %.2f' % rmse)

model = DecisionTreeRegressor(random_state=42)

MSE = []
RMSE = []

for i in range(len(X_train_list)):
    X_train = X_train_list[i]
    y_train = y_train_list[i]
    X_val = X_val_list[i]
    y_val = y_val_list[i]

    mse, rmse = train_predict(model, X_train, y_train, X_val, y_val)
    MSE.append(mse)
    RMSE.append(rmse)
plt.plot(new_dates['dates'], MSE, 'go--', linewidth=2, markersize=2)
plt.plot(new_dates['dates'], RMSE, 'bo-', linewidth=2, markersize=2)
plt.legend(('MSE', 'RMSE'),
        loc=(1, 1), handlelength=1.5, fontsize=12)
plt.xticks(rotation=45)
plt.show()


predictions = model.predict(np.array(test))

submission = pd.DataFrame({"fullVisitorId":test_ids})
predictions[predictions<0] = 0
submission["PredictedLogRevenue"] = predictions
submission = submission.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()
submission.columns = ["fullVisitorId", "PredictedLogRevenue"]
submission["PredictedLogRevenue"] = submission["PredictedLogRevenue"]
submission.to_csv("data/submission.csv", index=False)
print (submission.shape)
submission.head(10)

def shap_plot(model, X_val):

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_val)

    shap.summary_plot(shap_values, X_val)
i = -1
X_train = X_train_list[i]
y_train = y_train_list[i]
X_val = X_val_list[i]
y_val = y_val_list[i]
model = DecisionTreeRegressor(random_state=42).fit(X_train, y_train)
shap_plot(model, X_val)

SHAP (SHapley Additive exPlanations) is a unified approach to explain the output of any machine learning model. (https://github.com/slundberg/shap)

high pageviews contribute to a higher totalRevenue according to the SHAP plot
def feature_importance(model, X, y):
    perm = PermutationImportance(model, random_state=42).fit(X, y)
    return eli5.show_weights(perm, feature_names = X.columns.tolist())
feature_importance(model, X_val, y_val)

def plot_metrics(loss, val_loss):
    fig, (ax1) = plt.subplots(1, 1, sharex='col', figsize=(20,7))
    ax1.plot(loss, label='Train loss')
    ax1.plot(val_loss, label='Validation loss')
    ax1.legend(loc='best')
    ax1.set_title('Loss')
    plt.xlabel('Epochs')
    plt.show()
def plot_diff(X, diff):
    fig = plt.subplots(figsize=(18,6))
    plt.plot(diff, linewidth=1)
    plt.xticks(rotation=45)
    plt.show()
BATCH_SIZE = 64 # decrease if memory is not large enough
EPOCHS = 50
LEARNING_RATE = 1e-2

# Sandbox model creation
model = Sequential()
model.add(Dense(426, kernel_initializer='glorot_normal', input_dim=X_train_list[0].shape[1]))
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.4))
model.add(Dense(214, kernel_initializer='glorot_normal'))
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.3))
model.add(Dense(108, kernel_initializer='glorot_normal'))
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.2))
model.add(Dense(54, kernel_initializer='glorot_normal'))
model.add(LeakyReLU(alpha=0.01))
model.add(Dense(1))
Final model setup
model = Sequential()
model.add(Dense(256, kernel_initializer='glorot_normal', activation='relu', input_dim=X_train_list[0].shape[1]))
model.add(Dropout(0.4))
model.add(Dense(128, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dense(1))
# Setting a learning rate decay through the scheduler callback
# To be used in place of the decay rate in the optimizer
def schedule(epoch):
    print (LEARNING_RATE*(1/(epoch+1)))
    return LEARNING_RATE*(1/(epoch+1))

adam = optimizers.adam(lr=LEARNING_RATE, decay=0.001)
model.compile(loss='mse', optimizer=adam)

callback = [
  # Interrupt training if `val_loss` stops improving for so many epochs (patience) by an amount more than min_delta
  EarlyStopping(patience=4, monitor='val_loss', min_delta=0.0005),
  # Write TensorBoard logs to `./logs` directory
  TensorBoard(log_dir='./logs'),
  # Define a learning rate decay
#   LearningRateScheduler(schedule)
]

# To view Tensorboard 
# enter in command line  tensorboard --logdir='./logs/' and go to localhost:6006 in browser
Create additional callbacks

model.summary()

x = [4,3,2]
y = [1,1,1]
difference = np.subtract(x, y)
print (difference)
[3 2 1]
MSE = []
RMSE = []

for i in range(len(X_train_list)):
    X_train = np.array(X_train_list[i])
    y_train = np.array(y_train_list[i])
    X_val = np.array(X_val_list[i])
    y_val = np.array(y_val_list[i])

    history = model.fit(x=X_train, y=y_train, batch_size=BATCH_SIZE, epochs=EPOCHS,
                    verbose=1, callbacks=callback, validation_data=(X_val, y_val))

    val_predictions = model.predict(X_val)
    mse, rmse = performance_metric(val_predictions, y_val)
    MSE.append(mse)
    RMSE.append(rmse)

    difference = np.subtract(y_val, val_predictions)

    print('Model validation metrics')
    print ('Run {} of {}'.format(i+1, len(X_train_list)))
    print('MSE: %.2f' % mse)
    print('RMSE: %.2f' % rmse)
    plot_metrics(history.history['loss'], history.history['val_loss'])
    plot_diff(X_train, difference)



plt.plot(new_dates['dates'], MSE, 'go--', linewidth=2, markersize=2)
plt.plot(new_dates['dates'], RMSE, 'bo-', linewidth=2, markersize=2)
plt.legend(('MSE', 'RMSE'),
        loc=(1, 1), handlelength=1.5, fontsize=12)
plt.xticks(rotation=45)
plt.show()
predictions = model.predict(np.array(test))

submission = pd.DataFrame({"fullVisitorId":test_ids})
# submission = submission[:20000]
predictions[predictions<0] = 0
submission["PredictedLogRevenue"] = predictions
submission = submission.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()
submission.columns = ["fullVisitorId", "PredictedLogRevenue"]
submission["PredictedLogRevenue"] = submission["PredictedLogRevenue"]
submission.to_csv("data/submission.csv", index=False)
Shape of submission should be (617242, 2)

print (submission.shape)
submission.head(10)

model.save('models/model_11.h5')

model = models.load_model('models/model_9.h5')

